In [ ]:
import numpy as np
import pickle

In [ ]:
print("=== 1. 檢查 dictionary_indices.npz ===")
npz_path = '/home/tommy/Project/PcodeBERT/experiment/outputs/MAIE/dictionary_indices.npz'
data_npz = np.load(npz_path)
print("檔案內的 Keys:", data_npz.files)
for key in data_npz.files:
    print(f"Key [{key}] shape: {data_npz[key].shape}")

print(data_npz['src'])
print(data_npz['trg'])

In [ ]:
print("\n=== 2. 檢查 transformation_matrix_W.npy ===")
npy_path = '/home/tommy/Project/PcodeBERT/experiment/outputs/MAIE/transformation_matrix_W.npy'
data_npy = np.load(npy_path)
print("Shape:", data_npy.shape)
print("資料型態:", data_npy.dtype)
print("內容預覽 (前2列):\n", data_npy[:2])

In [ ]:

pkl_path = '/home/tommy/Project/PcodeBERT/experiment/outputs/corpus_arm_32.pkl'
with open(pkl_path, 'rb') as f:
    data_pkl = pickle.load(f)

print(data_pkl)

In [ ]:
import pickle

file = '/home/tommy/Project/PcodeBERT/experiment/outputs/data/arm_32/0a0eb3000fbe35d6c30028f245a8e4aab10f1cac9191998a741f9d9ca12041e1.pkl'


with open(file, "rb") as f:
    data = pickle.load(f)
    
print(data)

In [8]:
import numpy as np
from gensim.models import KeyedVectors

# 1. 設定路徑
npz_path = '/home/tommy/Project/PcodeBERT/experiment/outputs/MAIE/dictionary_indices.npz'
# 務必改成您實際的模型路徑
src_model_path = '/home/tommy/Project/PcodeBERT/experiment/model/Unimap/MAIE/skipgram_x86_64_vectors.kv'
trg_model_path = '/home/tommy/Project/PcodeBERT/experiment/model/Unimap/MAIE/skipgram_arm_32_vectors.kv'

# 2. 載入 Index 資料
print("Loading indices...")
data = np.load(npz_path)
src_indices = data['src']
trg_indices = data['trg']

# 3. 載入 Word2Vec 模型 (為了取得 index_to_key)
print("Loading Word2Vec models to retrieve vocabulary...")
src_model = KeyedVectors.load(src_model_path)
trg_model = KeyedVectors.load(trg_model_path)

# 4. 反查並印出前 20 個配對結果
print(f"\nTotal pairs found: {len(src_indices)}")
print("-" * 50)
print(f"{'Index':<10} | {'Source (x86)':<20} <-> {'Target (ARM)':<20} | {'Index':<10}")
print("-" * 50)

# 顯示前 20 個配對 (通常也是頻率最高的指令)
for i in range(min(20, len(src_indices))):
    s_idx = src_indices[i]
    t_idx = trg_indices[i]
    
    # 從模型詞彙表中取出指令字串
    s_word = src_model.index_to_key[s_idx]
    t_word = trg_model.index_to_key[t_idx]
    
    print(f"{s_idx:<10} | {s_word:<20} <-> {t_word:<20} | {t_idx:<10}")

Loading indices...
Loading Word2Vec models to retrieve vocabulary...

Total pairs found: 32722
--------------------------------------------------
Index      | Source (x86)         <-> Target (ARM)         | Index     
--------------------------------------------------
0          | CALL~0               <-> STMIB~R1~R3          | 4150      
1          | JZ~0                 <-> BEQ~0                | 2         
2          | JMP~0                <-> MOV~R8               | 186       
3          | JNZ~0                <-> BEQ~0                | 2         
4          | MOV~ESI~0            <-> MOV~R5~R3            | 288       
5          | XOR~EAX~EAX          <-> ADD~R3~R2            | 573       
6          | RET                  <-> LDMIA~R3~R0          | 613       
7          | MOV~EDX~0            <-> LDR~R10~[0]          | 249       
8          | MOV~RDI~RBP          <-> ADD~R8~R1            | 3160      
9          | MOV~EDI~0            <-> ADD~R8~R1            | 3160      
10         

In [9]:
import numpy as np
from gensim.models import KeyedVectors

# 1. 設定路徑
npz_path = '/home/tommy/Project/PcodeBERT/experiment/outputs/MAIE/dictionary_indices.npz'
src_model_path = '/home/tommy/Project/PcodeBERT/experiment/model/Unimap/MAIE/skipgram_x86_64_vectors.kv'
trg_model_path = '/home/tommy/Project/PcodeBERT/experiment/model/Unimap/MAIE/skipgram_arm_32_vectors.kv'

# 2. 載入資料
data = np.load(npz_path)
src_indices = data['src']
trg_indices = data['trg']
src_model = KeyedVectors.load(src_model_path)
trg_model = KeyedVectors.load(trg_model_path)

# 3. 計算並印出前 20 個配對的相似度
print(f"\nTotal pairs: {len(src_indices)}")
print("-" * 80)
print(f"{'Source (x86)':<30} | {'Target (ARM)':<30} | {'Score'}")
print("-" * 80)

for s_idx, t_idx in zip(src_indices[:20], trg_indices[:20]):
    # 取得詞彙
    s_word = src_model.index_to_key[s_idx]
    t_word = trg_model.index_to_key[t_idx]
    
    # 取得向量
    v_src = src_model[s_word]
    v_trg = trg_model[t_word]
    
    # 計算 Cosine Similarity
    score = np.dot(v_src, v_trg) / (np.linalg.norm(v_src) * np.linalg.norm(v_trg))
    
    print(f"{s_word:<30} | {t_word:<30} | {score:.5f}")

print("-" * 80)


Total pairs: 32722
--------------------------------------------------------------------------------
Source (x86)                   | Target (ARM)                   | Score
--------------------------------------------------------------------------------
CALL~0                         | STMIB~R1~R3                    | 0.35850
JZ~0                           | BEQ~0                          | 0.13991
JMP~0                          | MOV~R8                         | 0.17594
JNZ~0                          | BEQ~0                          | 0.12048
MOV~ESI~0                      | MOV~R5~R3                      | 0.35201
XOR~EAX~EAX                    | ADD~R3~R2                      | 0.16621
RET                            | LDMIA~R3~R0                    | 0.25853
MOV~EDX~0                      | LDR~R10~[0]                    | 0.17881
MOV~RDI~RBP                    | ADD~R8~R1                      | 0.42430
MOV~EDI~0                      | ADD~R8~R1                      | 0.34690
TEST~R

In [ ]:
prefixes = [
    # ("ADD", "ADD"),
    # ("SUB", "SUB"),
    # ("CMP", "CMP"),
    # ("AND", "AND"),
    # ("XOR", "ORR"), 
    # ("MOV", "MOV"),
    # ("JBE", "B"),  
    # ("PUSH", "PUSH")
    # ("LEA", "STR") 
]

print(f"\n{'Source (x86)':<40} | {'Target (ARM)':<40} | {'Score'}")
print("-" * 90)

for p_src, p_trg in prefixes:
    src_matches = [k for k in src_model.index_to_key if k.startswith(p_src)][:5000]
    trg_matches = [k for k in trg_model.index_to_key if k.startswith(p_trg)][:5000]

    if src_matches and trg_matches:
        for w_src, w_trg in zip(src_matches, trg_matches):
            v_src = src_model[w_src]
            v_trg = trg_model[w_trg]
            
            score = np.dot(v_src, v_trg) / (np.linalg.norm(v_src) * np.linalg.norm(v_trg))
            print(f"{w_src:<40} | {w_trg:<40} | {score:.5f}")
    else:
        print(f"{p_src + '...':<40} | {p_trg + '...':<40} | Not Found")
        
    print("-" * 90)


Source (x86)                             | Target (ARM)                             | Score
------------------------------------------------------------------------------------------
XOR~EAX~EAX                              | ORR~R3~R3                                | 0.16202
XOR~EDI~EDI                              | ORR~R3~R3~R2~<TAG>                       | 0.25289
XOR~EDX~EDX                              | ORR~R1~R1                                | 0.17344
XOR~ESI~ESI                              | ORR~R3~R3~R2                             | 0.11330
XOR~ECX~ECX                              | ORRNE~R3~R3                              | 0.09713
XOR~R12D~R12D                            | ORREQ~R3~R3                              | 0.15176
XOR~R8D~R8D                              | ORRS~R3~R2~R3                            | 0.15257
XOR~EBX~EBX                              | ORR~R2~R2                                | 0.12002
XOR~R9D~R9D                              | ORR~R0~R0            

In [6]:
import numpy as np

# 設定要測試的前綴
prefixes = ["ADD", "CMP", "XOR", "MOV", "JBE", "CBZ", "PUSH", "LEA", "AND"]
N_SRC = 20  # Source 取前 20 個
N_TRG = 100 # Target 取前 100 個

print(f"\n{'Source':<30} | {'Target':<30} | {'Score'}")
print("-" * 80)

for p in prefixes:
    # 篩選候選詞
    src_matches = [k for k in src_model.index_to_key if k.startswith(p)][:N_SRC]
    trg_matches = [k for k in trg_model.index_to_key if k.startswith(p)][:N_TRG]

    if src_matches and trg_matches:
        for w_src in src_matches:
            v_src = src_model[w_src]
            norm_src = np.linalg.norm(v_src)
            
            # 一個 Source 對應 多個 Target
            for w_trg in trg_matches:
                v_trg = trg_model[w_trg]
                score = np.dot(v_src, v_trg) / (norm_src * np.linalg.norm(v_trg))
                print(f"{w_src:<30} | {w_trg:<30} | {score:.5f}")
            
            # 每個 Source 結束後畫一條分隔線方便閱讀
            print("-" * 40)
    else:
        print(f"{p:<30} | Not Found")
        print("-" * 80)


Source                         | Target                         | Score
--------------------------------------------------------------------------------
ADD~RSP~0                      | ADD~R13~R13                    | 0.21328
ADD~RSP~0                      | ADD~R12~R12                    | 0.22888
ADD~RSP~0                      | ADD~R3~R3                      | 0.22517
ADD~RSP~0                      | ADD~R0~R13                     | 0.22453
ADD~RSP~0                      | ADD~R3~R13                     | 0.17352
ADD~RSP~0                      | ADD~R1~R13                     | 0.15382
ADD~RSP~0                      | ADD~R1~R15~R1                  | 0.23243
ADD~RSP~0                      | ADD~R2~R13                     | 0.17625
ADD~RSP~0                      | ADD~R4~R4                      | 0.27359
ADD~RSP~0                      | ADD~R0~R15~R0                  | 0.20329
ADD~RSP~0                      | ADD~R0~R0                      | 0.21516
ADD~RSP~0                      |

In [7]:
# 指定的三個 Source 詞彙
target_sources = [
    "MOV~ESI~0",
    "ADDSD~<TAG>~<TAG>~<TAG>~[RSP+0]",
    "ANDPS~<TAG>~<TAG>~<TAG>~[0]"
]

print(f"\n{'Source':<40} | {'Target (Top 10 Matches)':<40} | {'Score'}")
print("-" * 95)

for w_src in target_sources:
    if w_src in src_model:
        # 取得 Source 向量
        v_src = src_model[w_src]
        
        # 使用 Target 模型計算與此向量最相似的所有詞並排序
        # topn=10 僅顯示前 10 名，若要更多可改為 20 或 50
        results = trg_model.similar_by_vector(v_src, topn=10)
        
        for w_trg, score in results:
            print(f"{w_src:<40} | {w_trg:<40} | {score:.5f}")
            
        print("-" * 95)
    else:
        print(f"{w_src:<40} | {'Not Found in Source Model':<40} | -")
        print("-" * 95)


Source                                   | Target (Top 10 Matches)                  | Score
-----------------------------------------------------------------------------------------------
MOV~ESI~0                                | MOV~R0                                   | 0.46045
MOV~ESI~0                                | LDRB~R0~[R10]~R7                         | 0.44802
MOV~ESI~0                                | SUBS~R2~R5                               | 0.43990
MOV~ESI~0                                | SUB~R2~R4~R7                             | 0.42036
MOV~ESI~0                                | BL~0                                     | 0.41571
MOV~ESI~0                                | CPYEQ~R12~R4                             | 0.41055
MOV~ESI~0                                | CPYNE~R5~R1                              | 0.40325
MOV~ESI~0                                | BNE~0                                    | 0.40267
MOV~ESI~0                                | LDRNE~R5~[0]    

In [1]:
from gensim.models import Word2Vec

# 設定你的檔案名稱
src_model_path = "skipgram_x86_64.model"
trg_model_path = "skipgram_arm_32.model"

# 輸出檔案名稱 (這才是之後要丟給 vecmap 的)
src_output_txt = "x86_64.vec"
trg_output_txt = "arm_32.vec"

print(f"Loading {src_model_path}...")
model_src = Word2Vec.load(src_model_path)
# binary=False 重點！這會存成純文字檔
model_src.wv.save_word2vec_format(src_output_txt, binary=False)
print(f"Saved to {src_output_txt}")

print(f"Loading {trg_model_path}...")
model_trg = Word2Vec.load(trg_model_path)
model_trg.wv.save_word2vec_format(trg_output_txt, binary=False)
print(f"Saved to {trg_output_txt}")

Loading skipgram_x86_64.model...


FileNotFoundError: [Errno 2] No such file or directory: 'skipgram_x86_64.model'